This Notebook runs simple one-dimensional FUNWAVE simulations. 
-----

It consists of the following four steps:

* **Step #1** - Bathymetry: In this step you can generate the one-dimensional bathymetry that will be used in the FUNWAVE simulation. The user has the option of creating a flat bathymetry, a slope bathymetry or upload his own file. Once the depth file is generated, proceed to Step #2.

* **Step #2** - Input File: In this tab you can generate the Input file with your desired specifications for the simulation. After generating your input file, proceed to Step #3.

* **Step #3** - Run Funwave: This tab lets you run FUNWAVE without the need of a terminal. You can also see the simulation's proggress and abort it if it is necessary. Proceed to Step #4 to process your results.

* **Step #4** - Post Processing: This tab gives you the option of generating an image at an specific time or generate a video of the simulation.

In [1]:
import pyFiles.main1D

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget